In [120]:
import sys
import os
import re
import pprint

In [221]:
email_first_pat = '([\w\.]+)\s?(?:@|\sat\s)\s?([\w\.\s]+)[\.\s](?:edu|EDU)'

In [222]:
#match "engler WHERE stanford DOM edu"
email_second_pat = '([\w]+)\sWHERE\s([\w]+)\sDOM\sedu'

In [223]:
#match "jks at robotics;stanford;edu"
email_third_pat = '([\w]+)\sat\s([\w;]+)edu'

In [235]:
#match "obfuscate('stanford.edu','jurafsky')"
email_forth_pat = "obfuscate\('([\w\.]+)','([\w\.]+)'\)"

In [225]:
#match "(650)725-3726"
phone_first_pat = '\(([0-9]{3})\)\s?([0-9]{3})\-([0-9]{4})'

In [226]:
#match "650-725-3726"
phone_second_pat = '([0-9]{3})\-([0-9]{3})\-([0-9]{4})'

In [227]:
#match "650 725 9046", "650 725-9046"
phone_third_pat = '([0-9]{3})\s([0-9]{3})[\s\-]([0-9]{4})'

In [228]:
def exchange(m):
    if 'edu' in m[0]:
        m = tuple([m[1], m[0].replace('.edu', '')])
    return m

In [230]:
email_pat = '|'.join([email_first_pat, email_second_pat, email_third_pat, email_forth_pat])
phone_pat = '|'.join([phone_first_pat, phone_second_pat, phone_third_pat])

In [233]:
"""
TODO
This function takes in a filename along with the file object (actually
a StringIO object at submission time) and
scans its contents against regex patterns. It returns a list of
(filename, type, value) tuples where type is either an 'e' or a 'p'
for e-mail or phone, and value is the formatted phone number or e-mail.
The canonical formats are:
     (name, 'p', '###-###-#####')
     (name, 'e', 'someone@something')
If the numbers you submit are formatted differently they will not
match the gold answers

NOTE: ***don't change this interface***, as it will be called directly by
the submit script

NOTE: You shouldn't need to worry about this, but just so you know, the
'f' parameter below will be of type StringIO at submission time. So, make
sure you check the StringIO interface if you do anything really tricky,
though StringIO should support most everything.
"""
def process_file(name, f):
    # note that debug info should be printed to stderr
    # sys.stderr.write('[process_file]\tprocessing file: %s\n' % (path))
    res = []
    for line in f:
        line = line.replace(' dot ', '.')
        line2 = line.replace('-', '')
        matches = re.findall(email_pat,line2)
        for m in matches:
            m = tuple([mat.replace(';', '.').replace(' ', '.').rstrip('.') for mat in m if mat != ''])
            m = exchange(m)
            email = '%s@%s.edu' % m
            res.append((name,'e',email))
        matches = re.findall(phone_pat,line)
        for m in matches:
            m = tuple([mat for mat in m if mat != ''])
            phone = '%s-%s-%s' % m
            res.append((name,'p', phone))
    return res

"""
You should not need to edit this function, nor should you alter
its interface as it will be called directly by the submit script
"""
def process_dir(data_path):
    # get candidates
    guess_list = []
    for fname in os.listdir(data_path):
        if fname[0] == '.':
            continue
        path = os.path.join(data_path,fname)
        f = open(path,'r')
        f_guesses = process_file(fname, f)
        guess_list.extend(f_guesses)
    return guess_list

"""
You should not need to edit this function.
Given a path to a tsv file of gold e-mails and phone numbers
this function returns a list of tuples of the canonical form:
(filename, type, value)
"""
def get_gold(gold_path):
    # get gold answers
    gold_list = []
    f_gold = open(gold_path,'r')
    for line in f_gold:
        gold_list.append(tuple(line.strip().split('\t')))
    return gold_list

"""
You should not need to edit this function.
Given a list of guessed contacts and gold contacts, this function
computes the intersection and set differences, to compute the true
positives, false positives and false negatives.  Importantly, it
converts all of the values to lower case before comparing
"""
def score(guess_list, gold_list):
    guess_list = [(fname, _type, value.lower()) for (fname, _type, value) in guess_list]
    gold_list = [(fname, _type, value.lower()) for (fname, _type, value) in gold_list]
    guess_set = set(guess_list)
    gold_set = set(gold_list)

    tp = guess_set.intersection(gold_set)
    fp = guess_set - gold_set
    fn = gold_set - guess_set

    pp = pprint.PrettyPrinter()
    #print 'Guesses (%d): ' % len(guess_set)
    #pp.pprint(guess_set)
    #print 'Gold (%d): ' % len(gold_set)
    #pp.pprint(gold_set)
    print 'True Positives (%d): ' % len(tp)
    pp.pprint(tp)
    print 'False Positives (%d): ' % len(fp)
    pp.pprint(fp)
    print 'False Negatives (%d): ' % len(fn)
    pp.pprint(fn)
    print 'Summary: tp=%d, fp=%d, fn=%d' % (len(tp),len(fp),len(fn))

"""
You should not need to edit this function.
It takes in the string path to the data directory and the
gold file
"""
def main(data_path, gold_path):
    guess_list = process_dir(data_path)
    gold_list =  get_gold(gold_path)
    score(guess_list, gold_list)

In [234]:
main('../data/dev', '../data/devGOLD')

True Positives (112): 
set([('ashishg', 'e', 'ashishg@stanford.edu'),
     ('ashishg', 'e', 'rozm@stanford.edu'),
     ('ashishg', 'p', '650-723-1614'),
     ('ashishg', 'p', '650-723-4173'),
     ('ashishg', 'p', '650-814-1478'),
     ('balaji', 'e', 'balaji@stanford.edu'),
     ('bgirod', 'p', '650-723-4539'),
     ('bgirod', 'p', '650-724-3648'),
     ('bgirod', 'p', '650-724-6354'),
     ('cheriton', 'e', 'cheriton@cs.stanford.edu'),
     ('cheriton', 'e', 'uma@cs.stanford.edu'),
     ('cheriton', 'p', '650-723-1131'),
     ('cheriton', 'p', '650-725-3726'),
     ('dabo', 'e', 'dabo@cs.stanford.edu'),
     ('dabo', 'p', '650-725-3897'),
     ('dabo', 'p', '650-725-4671'),
     ('dlwh', 'e', 'dlwh@stanford.edu'),
     ('engler', 'e', 'engler@lcs.mit.edu'),
     ('engler', 'e', 'engler@stanford.edu'),
     ('eroberts', 'e', 'eroberts@cs.stanford.edu'),
     ('eroberts', 'p', '650-723-3642'),
     ('eroberts', 'p', '650-723-6092'),
     ('fedkiw', 'e', 'fedkiw@cs.stanford.edu'),
     